### The data examined here is fully available by following this [link](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/data) to Kaggle. 

In [ ]:
# import necessary libraries

import pandas as pd
import numpy as np
import os
import sys
import cv2
import PIL
from PIL import Image, ImageFile, Jpeg2KImagePlugin
import pickle


import imageio
from pydicom import dcmread
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# read in files and create path to image directories

train_df = pd.read_csv('train.csv')
train_img = ('train/')
test_img_path = ('test/')
ids = train_df.image_id

In [ ]:
# all null values are accounted for. Any 'class-name' that equals 'No finding', does not have any border box coordinates.
# Keep in Dataframe, as there is ambiguity with labeling.

train_df.info()

In [ ]:
# examine first rows of dataframe

train_df.head()

In [ ]:
# replace nan values in bounding box columns with zeros

train_df = train_df.fillna(0)

In [ ]:
# pull all values of image_id in dataframe. Note how there are over 67k image_ids but only 15000 unique ids

ids = train_df.image_id
unique_ids = train_df.image_id.unique()

print(len(ids))
print(len(unique_ids))

In [ ]:
# there are 17 different radiologists, split into groups of 3, who labeled each image

train_df.rad_id.unique()

In [ ]:
# overwhelming majority of images have 'No finding' measurement

train_df.class_name.value_counts()

In [ ]:
# visualize the data imbalance

plt.barh(train_df.class_name.value_counts(ascending=True).index, train_df.class_name.value_counts(ascending=True))
plt.title('Gross label count')
plt.xlabel('Observations')
plt.ylabel('Labeled Condition')
plt.show()

In [ ]:
# create a mask for each image, resize it, and scale each bounding box accordingly 

def im_mask(data, bb):
    rows, cols = data.shape
    Y = np.zeros((rows, cols))
    bb = bb.astype(np.int)
    Y[bb[0]:bb[2], bb[1]:bb[3]] = 1
    return Y
    
def mask_to_bb(Y):
    cols, rows = np.nonzero(Y)
    if len(cols) == 0:
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row])

def create_bb_array(df):
    return df.iloc[:,4:]

def resized(data,bb):
    resized = cv2.resize(im_mask(data, bb), (256,256))
    return mask_to_bb(resized)

In [ ]:
# initialize an empty dataframe to upload scaled data

emp_df = pd.DataFrame(index=range(0,len(train_df.image_id)), columns=['id', 'label', 'xmin', 'ymin', 'xmax', 'ymax'])
emp_df

In [ ]:
# for each unique image_id, read the dicom file, extract the pixel array then resize and flatten to be exported as a .npy
# file for easy reuse for modeling

ids = train_df.image_id.unique()
labels = []
df_data = []
boxes = []
new_boxes = []
count = 0
for n, id_ in enumerate(ids):
    dicom_path = train_img + id_ + '.dicom'
    dicom = pydicom.dcmread(dicom_path)
    
    data = dicom.pixel_array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    im = Image.fromarray(data)
    
    new_im = im.resize((256,256))
    npdata = np.asarray(new_im)
    
    npdata = npdata.flatten().reshape(1, 65536)
    df_data.append(npdata)

    df = train_df[train_df.image_id == id_]
    box = df.iloc[:,4:].values
    label = df.iloc[:,2].values
    labels.append(label)
    boxes.append(box)

    for l,b in zip(labels,boxes):
        for x,y in zip(l,b):
            new_bb = resized(data,y)
            emp_df.iloc[count,0] = id_
            emp_df.iloc[count,1] = x
            emp_df.iloc[count,2:] = new_bb  
            count +=1
    boxes = []
    labels = []  

In [ ]:
# export populated dataframe to csv file for modeling exercises 

emp_df.to_csv('data.csv')

In [ ]:
# open path to test images directory for pixel extraction and manipulation

test_img_list = os.listdir(test_img_path)

In [ ]:
# pull each image file from the directory, extract pixel array, resize, flatten and append it to a list for exporting

test_img_ar = []

for t_img in test_img_list:
    dicom = pydicom.dcmread(test_img_path + t_img)
    
    data = dicom.pixel_array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    im = Image.fromarray(data)
    
    new_im = im.resize((256,256))
    test_data = np.asarray(new_im)
    
    test_data = test_data.flatten().reshape(1, 65536)
    test_img_ar.append(test_data)

In [ ]:
# save numpy arrays of train images as a numpy zip file

np.savez('arrays.npz', *df_data)

In [ ]:
# save numpy arrays of test images as a numpy zip file

np.savez('test_arrays.npz', *test_img_ar)

In [ ]:
# create a single list of lists to iterate through

file = zip(lab_id, labels)
        

In [ ]:
# Iterate through the zipped list to find image ids with the highest value count of of each possible class label

for x,y in file:
    for l in y:
        a = x
        b = l.index
        c = l[0]
        if b[0] == 'Pulmonary fibrosis':
            if c > 5:
                print(a, l)

In [ ]:
# list of labels

lab_list = [x for x in train_df.class_name.unique()]

In [ ]:
# images with the highest concentration of labeling for each class
### note that 'No finding' images have a limit of 3 labels: a single label from each of the three radiologists, so all 
### 'No finding' images had identical class labels

high_ims = {'Cardiomegaly': 'd61eb45d47ad48020286203b1f1362f8', 
            'Aortic enlargement': 'e82620b01bbc77792885029d3cd0d8ae', 
            'Pleural thickening': 'e31be972e181987a8600a8700c1ebe88', 'ILD': 'd3823d24855b6ef03c188e962948b4b9', 
            'Nodule/Mass': '03e6ecfa6f6fb33dfeac6ca4f9b459c9', 'PF':'e62c07fde352cc658af3f989fe0b546f', 
            'Lung Opacity': '4068af795c7cb80fec0883dab82f4fbf', 'Atelectasis': '1dafb16f8c69e188cf2152200e0cb2ef', 
            'Other lesion': '53b1a490cd7e3a30e94014bdfd314d14', 'Infiltration': '1aaa4b217affae30113bd3a7a384a4c7', 
            'Pleural effusion': '04bb8bd7ee6f88a16623fe5c6dd4da91', 'Calcification': 'dfd523a5991fc852654bf1235c6282c6', 
            'Consolidation': '4b91d54f3170a9c8a757e6acd6c25588', 'Pneumothorax': 'f51434ef988e30a05f8b0986814d9485'}

In [ ]:
# Pull images from dataframe that have highest frequency of one label name, create images from their pixel arrays, and plot 
# all bounding boxes linked to that image

### Important step to take because it visualizes the common 'look' of a condition versus another. Not only that, some 
### conditions, like 'Nodule/Mass' rarely has one to five boxes labeled, but upwards of 20+.
### Visualizing each condition also provides an insight as to how the model will view each image. For example, since the 
### 'Nodule/Mass' class is most frequently labeled upwards of 20 times, its pixel array will have a signature that 
### will correspond to that trend. 

ims = []
bbox = []
for key,id_ in high_ims.items():

    dicom_path = train_img + id_ + '.dicom'
    dicom = pydicom.dcmread(dicom_path)
    print(dicom)
    data = dicom.pixel_array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    im = Image.fromarray(data)
    
    df = train_df[train_df.image_id == id_]
    box_values = df.iloc[:,4:]
    for v in box_values.values:
        xy = (v[0], v[1])
        width = (v[2] - v[0])
        height = (v[3] - v[1])
        box = patches.Rectangle((xy), width=width, height=height, edgecolor='white', fill=False)
        bbox.append(box)
    fig, ax = plt.subplots(1)
    plt.figure(figsize=(20,20))
    for b in bbox:
        ax.add_patch(b)
    ax.set_title(str(key))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(im)
    bbox = []